We will set up everything as we did in tutorial 1 except we no longer need sw_build_systems and instead now need to work with sw_openmm

In [ ]:
from modules.sw_build_systems import *
from modules.sw_openmm import *

In [ ]:
main_dir = !pwd
main_dir = main_dir[0]
directories = PolymerSimulatorDirs(main_dir)

We will define our sw_openmm object as 'simulation' and use this to build and run simulations.